In [ ]:
'''
Generate panels for figure 10

Maximilian Eggl
Nov 2024
'''


import sys

from ExtFuncs import *
%load_ext autoreload
%autoreload 2

import scipy.stats as stats

plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('ytick', labelsize=28) 
plt.rc('xtick', labelsize=28)

import networkx as nx
import networkx.algorithms.community as nx_comm
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import ward, fcluster
from scipy.spatial.distance import squareform,pdist

import matplotlib.patches as patches
import matplotlib.colors as mcolors

plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

from statsmodels.stats.multitest import multipletests

In [ ]:
datDir = './Dat/'
ColorCode = {'D1Nac':np.array([117/255,20/255,124/255]),'D2Nac':np.array([241/255,164/255,84/255]),
             'mPFC':np.array([86/255,187/255,126/255]),'Sus':np.array((0.9682, 0.7208441, 0.612293)),
            'Res':np.array((0.705673158, 0.01555616, 0.150232812)),'nForg':np.array((0.6672529243333334, 0.779176457, 0.992959213)),
            'Forg':np.array((0.2298057, 0.298717966, 0.753683153))}

FigLabs = ['Mean degree \n [edges per node]','Efficiency','Modularity','Betweeness','Assortativity']

def PlotSig(left,right,height,starAdjust=0.02,col = 'k',ax=None, text = None):
    if(ax is None):
        plt.plot([left,right],[height]*2,c=col)
        if(text is None):
            plt.plot((left+right)/2,height+starAdjust,marker=(6, 2,0 ),ms=10,c=col)
        else:
            plt.text((left+right)/2,height+starAdjust,text,ha='center')
    else:
        ax.plot([left,right],[height]*2,c=col)
        if(text is None):
            ax.plot((left+right)/2,height+starAdjust,marker=(6, 2,0 ),ms=10,c=col)
        else:
            ax.text((left+right)/2,height+starAdjust,text,ha='center')   

def PlotViolScat(A,col,pos,lab=None,ax= None,hatch=False):
    if(ax is None):
        vp = plt.violinplot(A,
                positions=[pos],widths=0.2,showmeans='True'
                )  
        vp['bodies'][0].set_facecolor(col)
        vp['cbars'].set_color(col)
        vp['cmins'].set_color(col)
        vp['cmaxes'].set_color(col)
        vp['cmeans'].set_color('black')
        plt.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
        if(hatch):
            vp['bodies'][0].set_hatch('//')
    else:
        vp = ax.violinplot(A,
                positions=[pos],widths=0.2,showmeans='True'
                )  
        vp['bodies'][0].set_facecolor(col)
        vp['cbars'].set_color(col)
        vp['cmins'].set_color(col)
        vp['cmaxes'].set_color(col)
        vp['cmeans'].set_color('black')
        if(hatch):
            vp['bodies'][0].set_hatch('//')
        ax.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)

In [ ]:
def GetGraphMetrics(G,labels):
    #degree mean
    mdd = np.mean(sorted((d for n, d in G.degree()), reverse=True))
    #global efficiency
    ge  = nx.global_efficiency(G)
    #betweenness
    be  = np.mean(list(nx.betweenness_centrality(G,normalized=True).values()))
    #modularity
    try: #this messes stuff up - without try gets the figures in E
        t = [np.argwhere(labels==i).squeeze() for i in range(1,max(labels)+1)]
        mod = nx_comm.modularity(G, t)
    except:
        mod = 0
    #assortativity
    ass = nx.degree_pearson_correlation_coefficient(G)
    return mdd,ge,be,mod,ass
def GetGPIO(datDir):
    for file in os.listdir(datDir):
        if file.endswith(".csv"):
            GDat = pd.read_csv(datDir+'/'+file)
            G1 = GDat[GDat[' Channel Name'] == ' GPIO-1']
            G2 = GDat[GDat[' Channel Name'] == ' GPIO-2']
            G3 = GDat[GDat[' Channel Name'] == ' GPIO-3']
            try:
                Tmax = G1['Time (s)'].max()
            except Exception as e:
                print(e)
                EndTime,StartTime,Tmax = math.nan,math.nan,math.nan
            return Tmax,G2,G3
def GetDat_loc(Batch,Expts,datDir,FreezeTime=50,eFlag = 0):
    print('='*20)
    print('Doing '+Batch)
    DatNames  = []
    Dat       = []
    Times     = []
    ShockTimes = []
    TimeCalcDat = []
    CalcDat = []
    RawDat  = []
    Lights  = []
    for file in tqdm(os.listdir(datDir+Batch)):
        if os.path.isdir(datDir+Batch+'/'+file):    
            newDir = datDir+Batch+'/'+file+'/'
            DatNames.append(file)
            T = []
            d = []
            st = []
            litLights = []
            for file2 in np.sort(os.listdir(newDir)):
                if(os.path.isdir(newDir+file2) and file2+'/' in Expts):
                    t,g2,g3 = GetGPIO(newDir+file2+'/')
                    T.append(t)
                    litLights.append([g2,g3])
            Lights.append(litLights)
            MatDir =datDir+Batch+'/'+file+'/Data_Miniscope_PP.mat'
            NDat = read_mat(MatDir)
            sDat,cDat,cRaw = PrepData(NDat)
            CalcDat.append([cDat[0],cDat[-2]]) 
            RawDat.append([cRaw[0],cRaw[-2]]) 
            lens = [len(cDat[0][0]),len(cDat[-2][0])]
        
            TimeCalcDat.append([np.linspace(0,tm,l) for tm,l in zip(T,lens)])
    return Dat,np.array(Times),DatNames,TimeCalcDat,CalcDat,RawDat,Lights

In [ ]:
DatA,TimesA,DatNamesA,TimeCalcDatA,CalcDatA,ShockTimesA,cRawA = GetDat('Batch A 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)
DatB,TimesB,DatNamesB,TimeCalcDatB,CalcDatB,ShockTimesB,cRawB = GetDat('Batch B 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)

DatMPC = DatA+DatB
TimesMPC = np.vstack([TimesA,TimesB])
DatNamesMPC = DatNamesA + DatNamesB
TimeCalcDatMPC = TimeCalcDatA+TimeCalcDatB
CalcDatMPC = CalcDatA+CalcDatB
ShockTimesMPC = np.vstack([ShockTimesA,ShockTimesB])
labelsMPC = np.array([1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1])

In [ ]:
FullOrder = []
FullOrderA = []
i=0
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    TC = TimeCalcDatMPC[j][i]
    ST = ShockTimesMPC[j][i]
    Order = np.zeros(Calc.shape[0])
    Order2 = np.zeros(Calc.shape[0])
    Fire0 = Calc[:,TC<ST[0]].mean(axis=1)==0.0
    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order = np.zeros_like(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    FullOrder.append(Order)
    FullOrderA.append(Order2)
    
print('-'*10 + 'Done 1'+'-'*10)    
i = 1
FullOrder2 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrder[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderA[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder2.append(Order)
print('-'*10 + 'Done 2'+'-'*10)
i = 2
FullOrder3 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder3.append(Order)
print('-'*10 + 'Done 3'+'-'*10)
i = 3
FullOrder4 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder3[j]<0,FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder3[j]>0,FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder4.append(Order)
print('-'*10 + 'Done 4'+'-'*10)
TotalOrder = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrder,FullOrder2,FullOrder3,FullOrder4)]

In [ ]:
Calc =  CalcDatMPC[3][0]
Calc = Calc[Calc.sum(axis=1)>0]
O = FullOrder[3] + FullOrderA[3]
O = O[~np.isnan(O)]
Calc = Calc[np.argsort(O)]
M = np.zeros((len(Calc),len(Calc)))
for i,C1 in enumerate(Calc):
    for j,C2 in enumerate(Calc):
        M[i,j] = np.corrcoef(C1,C2)[0,1]

In [ ]:

normalize = mcolors.Normalize(vmin=-0.3, vmax=1)
plt.imshow(M,norm=normalize)
rect = patches.Rectangle((0, 0), sum(O<0), sum(O<0), linewidth=3, edgecolor=ColorCode['D1Nac'], 
                         facecolor='none')
ax = plt.gca()
ax.add_patch(rect)
rect = patches.Rectangle((sum(O<0), sum(O<0)), sum(O==0), sum(O==0),  linewidth=3, edgecolor='gray',  facecolor='none')
ax.add_patch(rect)
rect = patches.Rectangle((sum(O<1), sum(O<1)), sum(O==1)-0.8, sum(O==1)-0.8,  linewidth=3, edgecolor=ColorCode['D1Nac']+0.3,  facecolor='none')
ax.add_patch(rect)
cbar = plt.colorbar(ax=ax, location='left')
cbar.set_label('Correlation',fontsize=32,rotation=90,labelpad=10)
cbar.set_ticks([-0.2,0,0.2,0.4,0.6,0.8,1.0])
plt.xticks([])
plt.yticks([])
plt.ylabel('Neurons',fontsize=28)
plt.xlabel('Neurons',fontsize=28)

In [ ]:
np.fill_diagonal(M,0)
M[abs(M)<0.3] = 0
normalize = mcolors.Normalize(vmin=-0.3, vmax=1)
G = nx.from_numpy_array(M)
edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
#p = nx.spring_layout(G,k=0.75, iterations=20)
p = nx.circular_layout(G)

theta = 0.6*np.pi / 4  # Rotate by 45 degrees as an example. Adjust as needed.

rotation_matrix = np.array([[np.cos(theta), -np.sin(theta)], 
                            [np.sin(theta),  np.cos(theta)]])

# Apply rotation to each position
#for node in p:
#    p[node] = (-p[node][0], p[node][1])  # Negate the x-coordinate to mirror around the y-axis
for node, pos in p.items():
    p[node] = np.dot(rotation_matrix, pos)

fig,ax = plt.subplots()
label_colors = {-2: ColorCode['D1Nac'],-1: ColorCode['D1Nac'], 0: 'gray', 1: ColorCode['D1Nac']+0.3}

node_colors = [label_colors[np.sort(O)[node]] for node in G.nodes()]

nx.draw(G,p,node_color=node_colors,node_size=20, edgelist=edges, edge_color=weights, width=2.0, 
               vmin = -0.3,vmax = 1)
sm = plt.cm.ScalarMappable(cmap=plt.cm.inferno, 
                           norm=plt.Normalize(-0.3,vmax = 1))

ax.set_aspect('equal')

In [ ]:
DA,TA,DNA,TCA,CA,RA,LA = GetDat_loc('Batch A 2022 mPFC',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)
DB,TB,DNB,TCB,CB,RB,LB = GetDat_loc('Batch B 2022 mPFC',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)


DAH = DA+DB
TH = np.vstack([TA,TB])
DNH = DNA + DNB
TCH = TCA+TCB
CH = CA+CB
LH = LA+LB

In [ ]:
premdd   = []
prenx = []
prebe = []
preass   = []
postmdd   = []
postnx = []
postbe = []
postass   = []
for i,C2 in tqdm(enumerate(CH)):
        C1 = C2[0]
        Calc = C1[C1.sum(axis=1)>0]
        M = np.corrcoef(C1)
        np.fill_diagonal(M,0)
        M[abs(M)<0.3] = 0
        G = nx.from_numpy_array(M)
        premdd.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
        prenx.append(nx.global_efficiency(G))
        prebe.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
        preass.append(nx.degree_pearson_correlation_coefficient(G))
        C1 = C2[1]
        Calc = C1[C1.sum(axis=1)>0]
        M = np.corrcoef(C1)
        np.fill_diagonal(M,0)
        M[abs(M)<0.3] = 0
        G = nx.from_numpy_array(M)
        postmdd.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
        postnx.append(nx.global_efficiency(G))
        postbe.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
        postass.append(nx.degree_pearson_correlation_coefficient(G))
premddwo   = []
prenxwo = []
prebewo = []
preasswo   = []
postmddwo   = []
postnxwo = []
postbewo = []
postasswo   = []
for i,(C2,T) in tqdm(enumerate(zip(CH,TotalOrder))):
    C1 = C2[0][~T]
    Calc = C1[C1.sum(axis=1)>0]
    M = np.corrcoef(C1)
    np.fill_diagonal(M,0)
    M[abs(M)<0.3] = 0
    G = nx.from_numpy_array(M)
    premddwo.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
    prenxwo.append(nx.global_efficiency(G))
    prebewo.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
    preasswo.append(nx.degree_pearson_correlation_coefficient(G))
    C1 = C2[1][~T]
    Calc = C1[C1.sum(axis=1)>0]
    M = np.corrcoef(C1)
    np.fill_diagonal(M,0)
    M[abs(M)<0.3] = 0
    G = nx.from_numpy_array(M)
    postmddwo.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
    postnxwo.append(nx.global_efficiency(G))
    postbewo.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
    postasswo.append(nx.degree_pearson_correlation_coefficient(G))

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(16,4))
k=1
p1m = np.array(premdd)[labelsMPC==1]
p1p = np.array(premdd)[labelsMPC==2]
p2m = np.array(postmdd)[labelsMPC==1]
p2p = np.array(postmdd)[labelsMPC==2]
a=ax[0]

PlotViolScat(p1m,np.clip(ColorCode['D1Nac']+0.3,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['D1Nac'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['D1Nac']+0.3,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['D1Nac'],0,1),2.2,ax=a,hatch=True)

a=ax[1]
p1m = np.array(preass)[labelsMPC==1]
p1p = np.array(preass)[labelsMPC==2]
p2m = np.array(postass)[labelsMPC==1]
p2p = np.array(postass)[labelsMPC==2]


if(stats.ttest_ind(p2p,p1p).pvalue<0.1):
    if(stats.ttest_ind(p2p,p1p).pvalue<0.05):
        PlotSig(k+1.2,k+1,0.7*max((np.max(p1p),np.max(p2p))),ax=a)
    else:
        PlotSig(k+1.2,k+1,1.1*max((np.max(p1p),np.max(p2p))),ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2p,p1p).pvalue))
if(stats.ttest_ind(p2p,p2m).pvalue<0.1):
    if(stats.ttest_ind(p2p,p2m).pvalue<0.05):
        PlotSig(k+0.2,k+1.2,0.8*max((np.max(p2p),np.max(p2m))),ax=a)
    else:
        PlotSig(k+0.2,k+1.2,1.1*max((np.max(p2p),np.max(p2m))),ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2m,p2p).pvalue))

PlotViolScat(p1m,np.clip(ColorCode['D1Nac']+0.3,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['D1Nac'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['D1Nac']+0.3,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['D1Nac'],0,1),2.2,ax=a,hatch=True)

a=ax[2]
p1m = np.array(prebe)[labelsMPC==1]
p1p = np.array(prebe)[labelsMPC==2]
p2m = np.array(postbe)[labelsMPC==1]
p2p = np.array(postbe)[labelsMPC==2]

if(stats.ttest_ind(p1m,p1p).pvalue<0.1):
    if(stats.ttest_ind(p1m,p1p).pvalue<0.05):
        a.plot(np.array([1,2]),[0.00031,0.00031],c='k')
        a.plot(np.array([1.45,1.55]),[0.00032,0.00032],marker=(6, 2,0 ),ms=10,c='k',ls='')
        #plt.plot(np.array([1,2,3])[m2]+0.03,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    else:
        PlotSig(k,k+1,1.1*max((np.max(p1p),np.max(p1m))),0.00001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p1p).pvalue))
if(stats.ttest_ind(p1m,p2m).pvalue<0.1):
    if(stats.ttest_ind(p1m,p2m).pvalue<0.05):
        PlotSig(k,k+0.2,1.2*max((np.max(p2m),np.max(p1m))),0.00001,ax=a)
    else:
        PlotSig(k,k+0.2,1.2*max((np.max(p2m),np.max(p1m))),0.00001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p2m).pvalue))
if(stats.ttest_ind(p2p,p1p).pvalue<0.1):
    if(stats.ttest_ind(p2p,p1p).pvalue<0.05):
        PlotSig(k+1.2,k+1,1.1*max((np.max(p1p),np.max(p2p))),0.00001,ax=a)
    else:
        PlotSig(k+1.2,k+1,1.1*max((np.max(p1p),np.max(p2p))),0.00001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2p,p1p).pvalue))
if(stats.ttest_ind(p2p,p2m).pvalue<0.1):
    if(stats.ttest_ind(p2p,p2m).pvalue<0.05):
        PlotSig(k+0.2,k+1.2,1.2*max((np.max(p2p),np.max(p2m))),0.00001,ax=a)
    else:
        PlotSig(k+0.2,k+1.2,1.2*max((np.max(p2p),np.max(p2m))),0.00001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2p,p2m).pvalue))
PlotViolScat(p1m,np.clip(ColorCode['D1Nac']+0.3,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['D1Nac'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['D1Nac']+0.3,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['D1Nac'],0,1),2.2,ax=a,hatch=True)

a=ax[3]
p1m = np.array(prenx)[labelsMPC==1]
p1p = np.array(prenx)[labelsMPC==2]
p2m = np.array(postnx)[labelsMPC==1]
p2p = np.array(postnx)[labelsMPC==2]
if(stats.ttest_ind(p1m,p1p).pvalue<0.1):
    if(stats.ttest_ind(p1m,p1p).pvalue<0.05):
        PlotSig(k,k+1,1*max((np.max(p1p),np.max(p1m))),0.001,ax=a)
    else:
        PlotSig(k,k+1,1*max((np.max(p1p),np.max(p1m))),0.001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p1p).pvalue))
if(stats.ttest_ind(p1m,p2m).pvalue<0.1):
    
    if(stats.ttest_ind(p1m,p2m).pvalue<0.05):
        PlotSig(k,k+0.2,1.02*max((np.max(p2m),np.max(p1m))),0.001,ax=a)
    else:
        PlotSig(k,k+0.2,1.02*max((np.max(p2m),np.max(p1m))),0.001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p2m).pvalue))
PlotViolScat(p1m,np.clip(ColorCode['D1Nac']+0.3,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['D1Nac'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['D1Nac']+0.3,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['D1Nac'],0,1),2.2,ax=a,hatch=True)
PlotSig(k+1.2,k+1,1.01*max((np.max(p1p),np.max(p2p))),0.002,ax=a,text="$p=${:.3f}".format(0.06844773))

proxy_artist1 = patches.Patch(color=np.clip(ColorCode['D1Nac']+0.3, 0, 1), label='Label for p1p')
proxy_artist2 = patches.Patch(facecolor='none', hatch='//', edgecolor=np.clip(ColorCode['D1Nac']+0.3, 0, 1), label='Label for p2m')

for a in ax:
    a.set_xticks([1.1,2.1],['R-','R+'])
ax[0].legend(handles=[proxy_artist1, proxy_artist2], labels=['HB1', 'HB2'])
ax[0].set_title('Mean degree \n [edges per node]')
ax[1].set_title('Assortativity')
ax[2].set_title('Betweenness')
ax[3].set_title('Efficiency')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=1, hspace=0.4)

In [ ]:
DatC,TimesC,DatNamesC,TimeCalcDatC,CalcDatC,ShockTimesC,cRawC = GetDat('Batch C 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=0)
DatD,TimesD,DatNamesD,TimeCalcDatD,CalcDatD,ShockTimesD,cRawD = GetDat('Batch D 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=0)

DatNAD1 = DatC+DatD
TimesNAD1 = np.vstack([TimesC,TimesD])
DatNamesNAD1 = DatNamesC + DatNamesD
TimeCalcDatNAD1 = TimeCalcDatC+TimeCalcDatD
CalcDatNAD1 = CalcDatC+CalcDatD
ShockTimesNAD1 = np.vstack([ShockTimesC,ShockTimesD])
RawDatNAD1 = cRawC+cRawD

In [ ]:
FullOrderD1Half = []
FullOrderD1HalfA = []
i=0
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    TC = TimeCalcDatNAD1[j][i]
    ST = ShockTimesNAD1[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrderD1Half.append(Order)
    FullOrderD1HalfA.append(Order2)

print('1 done')

i = 1
FullOrderD1Half2 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrderD1Half[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderD1HalfA[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD1Half2.append(Order)

print('2 done')

i = 2
FullOrderD1Half3 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD1Half2[j]<0,FullOrderD1Half[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD1Half2[j]>0,FullOrderD1HalfA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD1Half3.append(Order)
    
print('3 done')

i = 3
FullOrderD1Half4 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD1Half3[j]<0,FullOrderD1Half2[j]<0,FullOrderD1Half[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD1Half3[j]>0,FullOrderD1Half2[j]>0,FullOrderD1HalfA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD1Half4.append(Order)
    
print('4 done')

TotalOrderD1Half = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrderD1Half,FullOrderD1Half2,FullOrderD1Half3,FullOrderD1Half4)]

In [ ]:
DA,TA,DNA,TCA,CA,RA,LA = GetDat_loc('Batch C 2022 NAc',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)
DB,TB,DNB,TCB,CB,RB,LB = GetDat_loc('Batch D 2022 NAc',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)


DAH = DA+DB
TH = np.vstack([TA,TB])
DNH = DNA + DNB
TCH = TCA+TCB
CH = CA+CB
LH = LA+LB

In [ ]:
premdd   = []
prenx = []
prebe = []
preass   = []
postmdd   = []
postnx = []
postbe = []
postass   = []
for i,C2 in tqdm(enumerate(CH)):
        C1 = C2[0]
        Calc = C1[C1.sum(axis=1)>0]
        M = np.corrcoef(C1)
        np.fill_diagonal(M,0)
        M[abs(M)<0.3] = 0
        G = nx.from_numpy_array(M)
        premdd.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
        prenx.append(nx.global_efficiency(G))
        prebe.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
        preass.append(nx.degree_pearson_correlation_coefficient(G))
        C1 = C2[1]
        Calc = C1[C1.sum(axis=1)>0]
        M = np.corrcoef(C1)
        np.fill_diagonal(M,0)
        M[abs(M)<0.3] = 0
        G = nx.from_numpy_array(M)
        postmdd.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
        postnx.append(nx.global_efficiency(G))
        postbe.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
        postass.append(nx.degree_pearson_correlation_coefficient(G))
premddwo   = []
prenxwo = []
prebewo = []
preasswo   = []
postmddwo   = []
postnxwo = []
postbewo = []
postasswo   = []
for i,(C2,T) in tqdm(enumerate(zip(CH,TotalOrderD1Half))):
    C1 = C2[0][~T]
    Calc = C1[C1.sum(axis=1)>0]
    M = np.corrcoef(C1)
    np.fill_diagonal(M,0)
    M[abs(M)<0.3] = 0
    G = nx.from_numpy_array(M)
    premddwo.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
    prenxwo.append(nx.global_efficiency(G))
    prebewo.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
    preasswo.append(nx.degree_pearson_correlation_coefficient(G))
    C1 = C2[1][~T]
    Calc = C1[C1.sum(axis=1)>0]
    M = np.corrcoef(C1)
    np.fill_diagonal(M,0)
    M[abs(M)<0.3] = 0
    G = nx.from_numpy_array(M)
    postmddwo.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
    postnxwo.append(nx.global_efficiency(G))
    postbewo.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
    postasswo.append(nx.degree_pearson_correlation_coefficient(G))

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(16,4))
k=1
p1m = np.array(premdd)[labelsNAD1==1]
p1p = np.array(premdd)[labelsNAD1==2]
p2m = np.array(postmdd)[labelsNAD1==1]
p2p = np.array(postmdd)[labelsNAD1==2]
a=ax[0]

PlotViolScat(p1m,np.clip(ColorCode['D2Nac']+0.2,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['D2Nac'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['D2Nac']+0.2,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['D2Nac'],0,1),2.2,ax=a,hatch=True)

a=ax[1]
p1m = np.array(preass)[labelsNAD1==1]
p1p = np.array(preass)[labelsNAD1==2]
p2m = np.array(postass)[labelsNAD1==1]
p2p = np.array(postass)[labelsNAD1==2]
PlotSig(k,k+1,0.6*max((np.max(p1p),np.max(p1m))),ax=a)
PlotSig(k,k+0.2,0.7*max((np.max(p2m),np.max(p1m))),0.01,ax=a,text="$p=${:.2f}".format(0.05279631))

PlotViolScat(p1m,np.clip(ColorCode['D2Nac']+0.2,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['D2Nac'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['D2Nac']+0.2,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['D2Nac'],0,1),2.2,ax=a,hatch=True)

a=ax[2]
p1m = np.array(prebe)[labelsNAD1==1]
p1p = np.array(prebe)[labelsNAD1==2]
p2m = np.array(postbe)[labelsNAD1==1]
p2p = np.array(postbe)[labelsNAD1==2]


PlotViolScat(p1m,np.clip(ColorCode['D2Nac']+0.2,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['D2Nac'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['D2Nac']+0.2,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['D2Nac'],0,1),2.2,ax=a,hatch=True)

a=ax[3]
p1m = np.array(prenx)[labelsNAD1==1]
p1p = np.array(prenx)[labelsNAD1==2]
p2m = np.array(postnx)[labelsNAD1==1]
p2p = np.array(postnx)[labelsNAD1==2]
PlotSig(k,k+1,1.03*max((np.max(p1p),np.max(p1m))),0.01,ax=a)
PlotSig(k,k+0.2,1.01*max((np.max(p2m),np.max(p1m))),0.001,ax=a,text="$p=${:.2f}".format(0.056))
PlotViolScat(p1m,np.clip(ColorCode['D2Nac']+0.2,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['D2Nac'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['D2Nac']+0.2,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['D2Nac'],0,1),2.2,ax=a,hatch=True)


proxy_artist1 = patches.Patch(color=np.clip(ColorCode['D2Nac']+0.2, 0, 1), label='Label for p1p')
proxy_artist2 = patches.Patch(facecolor='none', hatch='//', edgecolor=np.clip(ColorCode['D2Nac']+0.2, 0, 1), label='Label for p2m')

for a in ax:
    a.set_xticks([1.1,2.1],['R-','R+'])
ax[2].legend(handles=[proxy_artist1, proxy_artist2], labels=['HB1', 'HB2'])
ax[0].set_title('Mean degree \n [edges per node]')
ax[1].set_title('Assortativity')
ax[2].set_title('Betweenness')
ax[3].set_title('Efficiency')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=1, hspace=0.4)

In [ ]:
DatC,TimesC,DatNamesC,TimeCalcDatC,CalcDatC,ShockTimesC,cRawC = GetDat('Batch E 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=0)
DatD,TimesD,DatNamesD,TimeCalcDatD,CalcDatD,ShockTimesD,cRawD = GetDat('Batch F 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=0)

DatNAD1 = DatC+DatD
TimesNAD1 = np.vstack([TimesC,TimesD])
DatNamesNAD1 = DatNamesC + DatNamesD
TimeCalcDatNAD1 = TimeCalcDatC+TimeCalcDatD
CalcDatNAD1 = CalcDatC+CalcDatD
ShockTimesNAD1 = np.vstack([ShockTimesC,ShockTimesD])
RawDatNAD1 = cRawC+cRawD

In [ ]:
FullOrderD1Half = []
FullOrderD1HalfA = []
i=0
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    TC = TimeCalcDatNAD1[j][i]
    ST = ShockTimesNAD1[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrderD1Half.append(Order)
    FullOrderD1HalfA.append(Order2)

print('1 done')

i = 1
FullOrderD1Half2 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrderD1Half[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderD1HalfA[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD1Half2.append(Order)

print('2 done')

i = 2
FullOrderD1Half3 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD1Half2[j]<0,FullOrderD1Half[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD1Half2[j]>0,FullOrderD1HalfA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD1Half3.append(Order)
    
print('3 done')

i = 3
FullOrderD1Half4 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD1Half3[j]<0,FullOrderD1Half2[j]<0,FullOrderD1Half[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD1Half3[j]>0,FullOrderD1Half2[j]>0,FullOrderD1HalfA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD1Half4.append(Order)
    
print('4 done')

TotalOrderD1Half = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrderD1Half,FullOrderD1Half2,FullOrderD1Half3,FullOrderD1Half4)]

In [ ]:
DA,TA,DNA,TCA,CA,RA,LA = GetDat_loc('Batch E 2022 NAc',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)
DB,TB,DNB,TCB,CB,RB,LB = GetDat_loc('Batch F 2022 NAc',['1.Habituation C2 Day-1/','6.Habituation C2/'],datDir)


DAH = DA+DB
TH = np.vstack([TA,TB])
DNH = DNA + DNB
TCH = TCA+TCB
CH = CA+CB
LH = LA+LB

In [ ]:
premdd   = []
prenx = []
prebe = []
preass   = []
postmdd   = []
postnx = []
postbe = []
postass   = []
for i,C2 in tqdm(enumerate(CH)):
        C1 = C2[0]
        Calc = C1[C1.sum(axis=1)>0]
        M = np.corrcoef(C1)
        np.fill_diagonal(M,0)
        M[abs(M)<0.3] = 0
        G = nx.from_numpy_array(M)
        premdd.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
        prenx.append(nx.global_efficiency(G))
        prebe.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
        preass.append(nx.degree_pearson_correlation_coefficient(G))
        C1 = C2[1]
        Calc = C1[C1.sum(axis=1)>0]
        M = np.corrcoef(C1)
        np.fill_diagonal(M,0)
        M[abs(M)<0.3] = 0
        G = nx.from_numpy_array(M)
        postmdd.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
        postnx.append(nx.global_efficiency(G))
        postbe.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
        postass.append(nx.degree_pearson_correlation_coefficient(G))
premddwo   = []
prenxwo = []
prebewo = []
preasswo   = []
postmddwo   = []
postnxwo = []
postbewo = []
postasswo   = []
for i,(C2,T) in tqdm(enumerate(zip(CH,TotalOrderD1Half))):
    C1 = C2[0][~T]
    Calc = C1[C1.sum(axis=1)>0]
    M = np.corrcoef(C1)
    np.fill_diagonal(M,0)
    M[abs(M)<0.3] = 0
    G = nx.from_numpy_array(M)
    premddwo.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
    prenxwo.append(nx.global_efficiency(G))
    prebewo.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
    preasswo.append(nx.degree_pearson_correlation_coefficient(G))
    C1 = C2[1][~T]
    Calc = C1[C1.sum(axis=1)>0]
    M = np.corrcoef(C1)
    np.fill_diagonal(M,0)
    M[abs(M)<0.3] = 0
    G = nx.from_numpy_array(M)
    postmddwo.append(np.mean(sorted((d for n, d in G.degree()), reverse=True)))
    postnxwo.append(nx.global_efficiency(G))
    postbewo.append(np.mean(list(nx.betweenness_centrality(G,normalized=True).values())))
    postasswo.append(nx.degree_pearson_correlation_coefficient(G))

In [ ]:
fig,ax = plt.subplots(1,4,figsize=(16,4))
k=1
p1m = np.array(premdd)[labelsNAD2==1]
p1p = np.array(premdd)[labelsNAD2==2]
p2m = np.array(postmdd)[labelsNAD2==1]
p2p = np.array(postmdd)[labelsNAD2==2]
a=ax[0]
if(stats.ttest_ind(p1m,p1p).pvalue<0.1):
    if(stats.ttest_ind(p1m,p1p).pvalue<0.05):
        PlotSig(k,k+1,1.1*max((np.max(p1p),np.max(p1m))),ax=a)
    else:
        PlotSig(k,k+1,1.1*max((np.max(p1p),np.max(p1m))),ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p1p).pvalue))
if(stats.ttest_ind(p1m,p2m).pvalue<0.1):
    if(stats.ttest_ind(p1m,p2m).pvalue<0.05):
        PlotSig(k,k+0.2,1.1*max((np.max(p2m),np.max(p1m))),ax=a)
    else:
        PlotSig(k,k+0.2,1.1*max((np.max(p2m),np.max(p1m))),ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p2m).pvalue))
if(stats.ttest_ind(p2p,p1p).pvalue<0.1):
    if(stats.ttest_ind(p2p,p1p).pvalue<0.05):
        PlotSig(k+1.2,k+1,1.1*max((np.max(p1p),np.max(p2p))),ax=a)
    else:
        PlotSig(k+1.2,k+1,1.1*max((np.max(p1p),np.max(p2p))),ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2p,p1p).pvalue))
if(stats.ttest_ind(p2p,p2m).pvalue<0.1):
    if(stats.ttest_ind(p2p,p2m).pvalue<0.05):
        PlotSig(k+0.2,k+1.2,1.1*max((np.max(p2p),np.max(p2m))),ax=a)
    else:
        PlotSig(k+0.2,k+1.2,1.1*max((np.max(p2p),np.max(p2m))),ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2m,p2p).pvalue))

PlotViolScat(p1m,np.clip(ColorCode['mPFC']+0.2,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['mPFC'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['mPFC']+0.2,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['mPFC'],0,1),2.2,ax=a,hatch=True)

a=ax[1]
p1m = np.array(preass)[labelsNAD2==1]
p1p = np.array(preass)[labelsNAD2==2]
p2m = np.array(postass)[labelsNAD2==1]
p2p = np.array(postass)[labelsNAD2==2]

if(stats.ttest_ind(p1m,p1p).pvalue<0.1):
    if(stats.ttest_ind(p1m,p1p).pvalue<0.05):
        PlotSig(k,k+1,0.6*max((np.max(p1p),np.max(p1m))),ax=a)
    else:
        PlotSig(k,k+1,0.6*max((np.max(p1p),np.max(p1m))),ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p1p).pvalue))
if(stats.ttest_ind(p1m,p2m).pvalue<0.1):
    if(stats.ttest_ind(p1m,p2m).pvalue<0.05):
        PlotSig(k,k+0.2,0.6*max((np.max(p2m),np.max(p1m))),ax=a)
    else:
        PlotSig(k,k+0.2,0.6*max((np.max(p2m),np.max(p1m))),0.01,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p2m).pvalue))
if(stats.ttest_ind(p2p,p1p).pvalue<0.1):
    if(stats.ttest_ind(p2p,p1p).pvalue<0.05):
        PlotSig(k+1.2,k+1,0.6*max((np.max(p1p),np.max(p2p))),ax=a)
    else:
        PlotSig(k+1.2,k+1,0.6*max((np.max(p1p),np.max(p2p))),ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2p,p1p).pvalue))
if(stats.ttest_ind(p2p,p2m).pvalue<0.1):
    if(stats.ttest_ind(p2p,p2m).pvalue<0.05):
        PlotSig(k+0.2,k+1.2,0.6*max((np.max(p2p),np.max(p2m))),ax=a)
    else:
        PlotSig(k+0.2,k+1.2,0.6*max((np.max(p2p),np.max(p2m))),ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2m,p2p).pvalue))

PlotViolScat(p1m,np.clip(ColorCode['mPFC']+0.2,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['mPFC'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['mPFC']+0.2,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['mPFC'],0,1),2.2,ax=a,hatch=True)

a=ax[2]
p1m = np.array(prebe)[labelsNAD2==1]
p1p = np.array(prebe)[labelsNAD2==2]
p2m = np.array(postbe)[labelsNAD2==1]
p2p = np.array(postbe)[labelsNAD2==2]

if(stats.ttest_ind(p1m,p1p).pvalue<0.1):
    if(stats.ttest_ind(p1m,p1p).pvalue<0.05):
        PlotSig(k,k+1,1.1*max((np.max(p1p),np.max(p1m))),0.00001,ax=a)
    else:
        PlotSig(k,k+1,1.1*max((np.max(p1p),np.max(p1m))),0.00001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p1p).pvalue))
if(stats.ttest_ind(p1m,p2m).pvalue<0.1):
    if(stats.ttest_ind(p1m,p2m).pvalue<0.05):
        PlotSig(k,k+0.2,1.2*max((np.max(p2m),np.max(p1m))),0.00001,ax=a)
    else:
        PlotSig(k,k+0.2,1.2*max((np.max(p2m),np.max(p1m))),0.00001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p2m).pvalue))
if(stats.ttest_ind(p2p,p1p).pvalue<0.1):
    if(stats.ttest_ind(p2p,p1p).pvalue<0.05):
        PlotSig(k+1.2,k+1,1.1*max((np.max(p1p),np.max(p2p))),0.00001,ax=a)
    else:
        PlotSig(k+1.2,k+1,1.1*max((np.max(p1p),np.max(p2p))),0.00001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2p,p1p).pvalue))
if(stats.ttest_ind(p2p,p2m).pvalue<0.1):
    if(stats.ttest_ind(p2p,p2m).pvalue<0.05):
        PlotSig(k+0.2,k+1.2,1.2*max((np.max(p2p),np.max(p2m))),0.00001,ax=a)
    else:
        PlotSig(k+0.2,k+1.2,1.2*max((np.max(p2p),np.max(p2m))),0.00001,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2p,p2m).pvalue))
PlotViolScat(p1m,np.clip(ColorCode['mPFC']+0.2,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['mPFC'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['mPFC']+0.2,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['mPFC'],0,1),2.2,ax=a,hatch=True)

a=ax[3]
p1m = np.array(prenx)[labelsNAD2==1]
p1p = np.array(prenx)[labelsNAD2==2]
p2m = np.array(postnx)[labelsNAD2==1]
p2p = np.array(postnx)[labelsNAD2==2]
if(stats.ttest_ind(p1m,p1p).pvalue<0.1):
    if(stats.ttest_ind(p1m,p1p).pvalue<0.05):
        PlotSig(k,k+1,1.07*max((np.max(p1p),np.max(p1m))),0.01,ax=a)
    else:
        PlotSig(k,k+1,1*max((np.max(p1p),np.max(p1m))),0.01,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p1p).pvalue))
if(stats.ttest_ind(p1m,p2m).pvalue<0.1):
    if(stats.ttest_ind(p1m,p2m).pvalue<0.05):
        PlotSig(k,k+0.2,1.02*max((np.max(p2m),np.max(p1m))),0.01,ax=a)
    else:
        PlotSig(k,k+0.2,1.02*max((np.max(p2m),np.max(p1m))),0.01,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p1m,p2m).pvalue))
if(stats.ttest_ind(p2p,p1p).pvalue<0.1):
    if(stats.ttest_ind(p2p,p1p).pvalue<0.05):
        PlotSig(k+1.2,k+1,1.02*max((np.max(p1p),np.max(p2p))),0.01,ax=a)
    else:
        PlotSig(k+1.2,k+1,1.02*max((np.max(p1p),np.max(p2p))),0.01,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2p,p1p).pvalue))
if(stats.ttest_ind(p2p,p2m).pvalue<0.1):
    if(stats.ttest_ind(p2p,p2m).pvalue<0.05):
        PlotSig(k+0.2,k+1.2,1*max((np.max(p2p),np.max(p2m))),0.01,ax=a)
    else:
        PlotSig(k+0.2,k+1.2,1*max((np.max(p2p),np.max(p2m))),0.01,ax=a,text="$p=${:.3f}".format(stats.ttest_ind(p2m,p2p).pvalue))
PlotViolScat(p1m,np.clip(ColorCode['mPFC']+0.2,0,1),1,ax=a)
PlotViolScat(p1p,np.clip(ColorCode['mPFC'],0,1),2,ax=a)

PlotViolScat(p2m,np.clip(ColorCode['mPFC']+0.2,0,1),1.2,ax=a,hatch=True)
PlotViolScat(p2p,np.clip(ColorCode['mPFC'],0,1),2.2,ax=a,hatch=True)


proxy_artist1 = patches.Patch(color=np.clip(ColorCode['mPFC']+0.2, 0, 1), label='Label for p1p')
proxy_artist2 = patches.Patch(facecolor='none', hatch='//', edgecolor=np.clip(ColorCode['mPFC']+0.2, 0, 1), label='Label for p2m')

for a in ax:
    a.set_xticks([1.1,2.1],['R-','R+'])
ax[2].legend(handles=[proxy_artist1, proxy_artist2], labels=['HB1', 'HB2'])
ax[0].set_title('Mean degree \n [edges per node]')
ax[1].set_title('Assortativity')
ax[2].set_title('Betweenness')
ax[3].set_title('Efficiency')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=1, hspace=0.4)